In [1]:
import sys
sys.path.append("../Tracking_SSD_ReID/")
from data_management.VOC2012ManagerObjDetection import VOC2012ManagerObjDetection
manager = VOC2012ManagerObjDetection("../data/VOCdevkit/VOC2012/")
imgs, boxes_gt, classes_gt = manager.getRawData(manager.batches[2][15:20])

import tensorflow as tf
from models.SSD300 import SSD300
SSD300_model = SSD300(21)
confs, locs = SSD300_model(imgs)

## Load SSD weights trained on VOC2012

In [2]:
SSD300_model.load_weights("../data/weights_alldata/ssd_weights_epoch_0380.h5")
SSD_backbone = SSD300_model.getVGG16()

## Import VGG16: the feature extractor of the segmentation model

In [3]:
from models.VGG16 import VGG16
VGG16 = VGG16(input_shape=(300, 300, 3))
VGG16_tilStage5 = tf.keras.models.Sequential([
    VGG16.getUntilStage4(),
    VGG16.getStage5()])

## Load weights from the SSD model

In [4]:
for i in range(len(VGG16_tilStage5.layers)):
    VGG16_tilStage5.get_layer(index=i).set_weights(
        SSD_backbone.get_layer(index=i).get_weights())